In [ ]:
import torch
import numpy as np
from omegaconf import OmegaConf

from physicsnemo.sym.models.fully_connected import FullyConnectedArch
from physicsnemo.sym.key import Key
from physicsnemo.sym.utils.io.vtk import var_to_polyvtk,grid_to_vtk

from three_fin_geometry import ThreeFin

# 1. 載入 config
cfg = OmegaConf.load("conf/conf_thermal.yaml")
cfg.run_mode = "eval"

# 2. 建立 input keys
input_keys = [
    Key("x"), Key("y"), Key("z"),
    Key("fin_height_s"),
    Key("fin_length_s"),
    Key("fin_thickness_s"),
]

# 3. 建立三個網路架構
flow_net = FullyConnectedArch(
    input_keys=input_keys, output_keys=[Key("u"), Key("v"), Key("w"), Key("p")]
)
thermal_f_net = FullyConnectedArch(input_keys=input_keys, output_keys=[Key("theta_f")])
thermal_s_net = FullyConnectedArch(input_keys=input_keys, output_keys=[Key("theta_s")])

# 4. 載入模型權重（指定路徑）
base_path = "/workspace/physicsnemo-sym/examples/three_fin_3d_0731_inf/outputs/three_fin_thermal/"

flow_net.load_state_dict(torch.load(base_path + "flow_network.0.pth", map_location="cpu"))
thermal_f_net.load_state_dict(torch.load(base_path + "thermal_f_network.0.pth", map_location="cpu"))
thermal_s_net.load_state_dict(torch.load(base_path + "thermal_s_network.0.pth", map_location="cpu"))

flow_net.eval()
thermal_f_net.eval()
thermal_s_net.eval()

# 5. 建立幾何 sample 點（使用你的參數）
geo = ThreeFin(parameterized=True)
param = {
    "fin_height_s": 0.030,
    "fin_length_s": 0.195,
    "fin_thickness_s": 0.0025,
}

In [ ]:
# 1️⃣ 流體區域推論
invar_fluid = geo.geo.sample_interior(nr_points=100000, parameterization=param, quasirandom=True)
invar_fluid_tensor = {k: torch.tensor(v, dtype=torch.float32) for k, v in invar_fluid.items()}
with torch.no_grad():
    pred_flow = flow_net(invar_fluid_tensor)
    pred_theta_f = thermal_f_net(invar_fluid_tensor)

pred_fluid_np = {
    **invar_fluid,
    **{k: v.cpu().numpy() for k, v in pred_flow.items()},
    **{k: v.cpu().numpy() for k, v in pred_theta_f.items()},
}

# 加入 velocity 欄位，用於 streamline
velocity = np.stack([
    pred_fluid_np["u"],
    pred_fluid_np["v"],
    pred_fluid_np["w"]
], axis=-1)
pred_fluid_np["velocity"] = velocity

# ➤ 輸出 fluid_only.vtp
var_to_polyvtk(pred_fluid_np, "fluid_only.vtp")

In [ ]:
from physicsnemo.sym.utils.io.vtk import VTKFromFile
from physicsnemo.sym.domain.inferencer  import PointVTKInferencer
import torch
from torch.utils.data import DataLoader, Dataset

import numpy as np
from sympy import Symbol, Eq, tanh, Or, And
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra.utils import to_absolute_path

import physicsnemo.sym
from physicsnemo.sym.hydra import to_absolute_path, instantiate_arch, PhysicsNeMoConfig
from physicsnemo.sym.utils.io import csv_to_dict
from physicsnemo.sym.solver import Solver
from physicsnemo.sym.domain import Domain
from physicsnemo.sym.geometry.primitives_3d import Box, Channel, Plane
from physicsnemo.sym.models.fourier_net import FourierNetArch
from physicsnemo.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)
from physicsnemo.sym.domain.monitor import PointwiseMonitor
from physicsnemo.sym.domain.inferencer import PointVTKInferencer
from physicsnemo.sym.utils.io import (
    VTKUniformGrid,
)
from physicsnemo.sym.key import Key
from physicsnemo.sym.node import Node
from physicsnemo.sym.eq.pdes.navier_stokes import NavierStokes
from physicsnemo.sym.eq.pdes.turbulence_zero_eq import ZeroEquation
from physicsnemo.sym.eq.pdes.basic import NormalDotVec, GradNormal
from three_fin_geometry import ThreeFin

In [ ]:

# add inferencer data
x, y, z = Symbol("x"), Symbol("y"), Symbol("z")

vtk_obj = VTKUniformGrid(
    bounds=[geo.geo_bounds[x], geo.geo_bounds[y], geo.geo_bounds[z]],
    npoints=[256, 128, 256],
    export_map={"u": ["u", "v", "w"], "p": ["p"]},
)

def mask_fn(x, y, z):
    # 強制變一維
    x = np.ravel(x)
    y = np.ravel(y)
    z = np.ravel(z)

    N = x.shape[0]

    # 補上 param 的每個輸入都 shape=(N,)
    invar = {
        "x": x,
        "y": y,
        "z": z,
        "fin_height_s": np.full((N,), param["fin_height_s"]),
        "fin_length_s": np.full((N,), param["fin_length_s"]),
        "fin_thickness_s": np.full((N,), param["fin_thickness_s"]),
    }

    # debug 用：可加上檢查 (可以之後註解掉)
    for k, v in invar.items():
        assert v.shape == (N,), f"{k} has shape {v.shape}, expected {(N,)}"

    sdf = geo.geo.sdf(invar, params=param)
    return sdf["sdf"] < 0



grid_inference = PointVTKInferencer(
    vtk_obj=vtk_obj,
    nodes=flow_net,
    input_vtk_map={"x": "x", "y": "y", "z": "z"},
    output_names=["u", "v", "w", "p"],
    mask_fn=mask_fn,
    mask_value=np.nan,
    requires_grad=False,
    batch_size=1000,
)

flow_domain.add_inferencer(grid_inference, "grid_inference")

In [ ]:
# 2️⃣ 固體區域推論
invar_solid = geo.three_fin.sample_interior(nr_points=50000, parameterization=param)
invar_solid_tensor = {k: torch.tensor(v, dtype=torch.float32) for k, v in invar_solid.items()}
with torch.no_grad():
    pred_theta_s = thermal_s_net(invar_solid_tensor)

pred_solid_np = {
    **invar_solid,
    **{k: v.cpu().numpy() for k, v in pred_theta_s.items()},
}
# 補齊空的 flow 變數，避免 paraview 出錯
for var in ["u", "v", "w", "p", "theta_f"]:
    pred_solid_np[var] = np.zeros_like(pred_solid_np["x"])
# ➤ 輸出 solid_only.vtp
var_to_polyvtk(pred_solid_np, "solid_only.vtp")

In [ ]:
import vtk
from vtk.util import numpy_support

def write_vtu(filename, coords, velocity, scalar_dict=None):
    n_points = coords.shape[0]

    # 點座標
    points = vtk.vtkPoints()
    for pt in coords:
        points.InsertNextPoint(*pt)

    # UnstructuredGrid
    grid = vtk.vtkUnstructuredGrid()
    grid.SetPoints(points)
    for i in range(n_points):
        vertex = vtk.vtkVertex()
        vertex.GetPointIds().SetId(0, i)
        grid.InsertNextCell(vertex.GetCellType(), vertex.GetPointIds())

    # 向量欄位：velocity
    velocity_array = numpy_support.numpy_to_vtk(velocity.reshape(-1, 3), deep=True)
    velocity_array.SetName("velocity")
    grid.GetPointData().AddArray(velocity_array)
    grid.GetPointData().SetVectors(velocity_array)

    # 額外 scalar 欄位（如 u,v,w,p 等）
    if scalar_dict:
        for key, val in scalar_dict.items():
            arr = numpy_support.numpy_to_vtk(val, deep=True)
            arr.SetName(key)
            grid.GetPointData().AddArray(arr)

    # 輸出 .vtu
    writer = vtk.vtkXMLUnstructuredGridWriter()
    writer.SetFileName(filename)
    writer.SetInputData(grid)
    writer.Write()
invar_fluid = {str(k): v for k, v in invar_fluid.items()}
pred_fluid_np = {
    **invar_fluid,
    **{k: v.cpu().numpy() for k, v in pred_flow.items()},
    **{k: v.cpu().numpy() for k, v in pred_theta_f.items()},
}
# ➤ 輸出 fluid_only.vtu for streamline
coords = np.stack([
    pred_fluid_np["x"],
    pred_fluid_np["y"],
    pred_fluid_np["z"]
], axis=-1)

scalar_dict = {
    "u": pred_fluid_np["u"],
    "v": pred_fluid_np["v"],
    "w": pred_fluid_np["w"],
    "p": pred_fluid_np["p"],
    "theta_f": pred_fluid_np["theta_f"],
}

write_vtu("fluid_only.vtu", coords, velocity, scalar_dict)


In [ ]:
!torchrun --nproc_per_node=2 three_fin_thermal.py
print(torch.cuda.memory_summary())